In [4]:
###################################################
# IMPORTANTE: Instalar la base de datos siguiendo #
# las instrucciones del README e indicar en estas #
# dos variables los direcctorios!!                #
TRAIN_REAL_DATA_PATH = "D:\Datos\TrainReal"
TRAIN_SIM_DATA_PATH  = "D:\Datos\TrainSimulation"
TEST_DATA_PATH       = "D:\Datos\TestReal"
###################################################
MODEL_PATH = "model"
OUTPUT_INFERENCE_PATH = "output"

import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from UAVIR_tools import BIRDSAIDataset, imgBoxes, createDir, genMovie

# Si se encuentra disponible, seleccionamos GPU como dispositivo para entrenar
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device: {}\n".format(device))

"""
Función que toma Faster-RCNN pre-entrenado y genera un nuevo modelo
aplicando un fine-tunning en el clasificador, adaptándolo al número
de clases, que debe de ser indicado en 'num_classes'. 
"""
def get_tunned_model(num_classes):
    # Cargamos el modelo
    model = fasterrcnn_resnet50_fpn()

    # Reemplazamos el clasificador de la red
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

"""
Función que nos retorna las transformaciones necesarias para codificar las
imágenes (en formato PIL), de tal forma que sean interpretables por la red.
"""
def get_transform():
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    return T.Compose(transforms) # __call__(<PIL image>)

Device: cuda



In [5]:
from torchvision import transforms as T

# Creamos el directorio donde vamos a guardar la salida
createDir(OUTPUT_INFERENCE_PATH)

# Transformaciones necesarias para que la entrada pueda
# ser interpretada por el modelo
transforms = get_transform()

# Seleccionamos el vídeo y el frame
data_path = TEST_DATA_PATH
movie_id = 11
frame = 100

dataset = BIRDSAIDataset(data_path, transforms)
frame_init, frame_end, n_frames = dataset.getMovieBoxInfo(movie_id)

# Recogemos la img del dataset
img, target = dataset[frame_init + frame]
images = [img.to(device)]

# Cargamos el modelo entrenado
num_classes = 3
model = get_tunned_model(num_classes)

model.to(device)
model.load_state_dict(torch.load(os.path.join(MODEL_PATH, "model_AUVIR_v0.3.pt")))

# Inferimos
model_eval = model.eval()
outputs = model_eval(images)

# Mostramos el resultado
FIGSIZE = [16, 9]
RES = 1920 # Full HD
RES = 2560 # 2k

transform_toImg = T.ToPILImage()
img = transform_toImg(img)

img_box_real = imgBoxes(img, target["boxes"], target["labels"])
img_box = imgBoxes(img, outputs[0]["boxes"], outputs[0]["labels"], outputs[0]["scores"], 0.2)

fig = plt.figure(figsize=FIGSIZE, dpi=RES/FIGSIZE[0])
ax1, ax2, ax3 = fig.subplots(1,3)
plt.subplots_adjust(wspace=0.01)

ax1.axis('off')
ax2.axis("off")
ax3.axis("off")

ax1.set_title("Imágen original")
ax2.set_title("Imágen original etiquetada")
ax3.set_title("Inferencia")

ax1.imshow(img)
ax2.imshow(img_box_real)
ax3.imshow(img_box)

if data_path == TRAIN_REAL_DATA_PATH:
    dataset_label = "TrainReal"
elif data_path == TEST_DATA_PATH:
    dataset_label = "TestReal"
elif data_path == TRAIN_SIM_DATA_PATH:
    dataset_label = "TrainSim"

file_name = "{}-{}_{}.png".format(dataset_label, str(movie_id).zfill(4), str(frame).zfill(5))

plt.savefig(os.path.join(OUTPUT_INFERENCE_PATH, file_name),
            bbox_inches='tight', pad_inches = 0.1)
plt.show()

¡El directorio 'output' ya existe!


TypeError: Expected input images to be of floating type (in range [0, 1]), but found type torch.uint8 instead